# Sentiment Analysis using Article Headlines from FinViz.com

In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [2]:
# Create a URL string and a list of tickers
finviz_url = "https://www.finviz.com/quote.ashx?t="
tickers = ["ZLAB"]

news_tables = {} 

# Loop through the tickers to combine with the URL to make one string representing a completed URL
for ticker in tickers:
    url = finviz_url + ticker
    
    # Request the data from URL (specify the user agent for headers)
    request = Request(url=url, headers={'user-agent': 'my-app'})
    response = urlopen(request)
    
    # Input the response data into BeautifulSoup specifying parse as html
    html = BeautifulSoup(response, 'html')
    
    # Get the HTML objects of the table that contain the article headlines and assign to each ticker in the dictionary
    news_table = html.find(id = 'news-table')
    news_tables[ticker] = news_table

In [3]:
parsed_data = []

for ticker, news_table in news_tables.items():
    
    # Getting all the tr elements from each row of the news_tables dictionary
    for row in news_table.findAll('tr'):
        # Get titles in the tag <a ...> 
        title = row.a.text
        # Split the text of date and time based on a space
        date_data = row.td.text.split(' ')
        # If the lenth of date_data is 1 then it is time. If the length is 2, first is date and second is time.
        if len(date_data) == 1:
            time = date_data[0]
        else:
            date = date_data[0]
            time = date_data[1]
        # Append the ticker, date, time, and title that results in a list of list inside the parsed_data
        parsed_data.append([ticker, date, time, title])

# print(parsed_data) 

In [4]:
# Import Pandas & Natural Language Toolkit Library
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [5]:
df = pd.DataFrame(parsed_data, columns=['ticker', 'date', 'time', 'title'])
df.head()

,ticker,date,time,title
0,ZLAB,Mar-12-21,11:54AM,7 Biotech Stocks With Catalysts That Go Far Be...
1,ZLAB,Mar-04-21,04:19PM,Why Amgen's $1.9 Billion Takeover Might Only W...
2,ZLAB,Mar-02-21,11:56PM,Analysts Are More Bearish On Zai Lab Limited (...
3,ZLAB,Mar-01-21,07:00AM,Zai Lab Announces Financial Results for Second...
4,ZLAB,Feb-27-21,09:23AM,15 Fastest Growing Biotech Companies in the US


In [6]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [7]:
f = lambda title: analyzer.polarity_scores(title)['compound']
df['compound'] = df['title'].apply(f)
df['date'] = pd.to_datetime(df.date).dt.date
df.head()

,ticker,date,time,title,compound
0,ZLAB,2021-03-12,11:54AM,7 Biotech Stocks With Catalysts That Go Far Be...,0.0000
1,ZLAB,2021-03-04,04:19PM,Why Amgen's $1.9 Billion Takeover Might Only W...,0.0000
2,ZLAB,2021-03-02,11:56PM,Analysts Are More Bearish On Zai Lab Limited (...,-0.2263
3,ZLAB,2021-03-01,07:00AM,Zai Lab Announces Financial Results for Second...,0.0000
4,ZLAB,2021-02-27,09:23AM,15 Fastest Growing Biotech Companies in the US,0.1779


In [8]:
df.to_csv("../Data/zlab_sentiment_finviz.csv", index=False)

In [9]:
# Sentiment calculation based on compound score
def get_sentiment(score):
    """
    Calculates the sentiment based on the compound score.
    """
    result = 0  # Neutral by default
    if score >= 0.05:  # Positive
        result = 1
    elif score <= -0.05:  # Negative
        result = -1

    return result

In [11]:
df.drop(columns=["compound"], inplace=True)
df.head()

,ticker,date,time,title
0,ZLAB,2021-03-12,11:54AM,7 Biotech Stocks With Catalysts That Go Far Be...
1,ZLAB,2021-03-04,04:19PM,Why Amgen's $1.9 Billion Takeover Might Only W...
2,ZLAB,2021-03-02,11:56PM,Analysts Are More Bearish On Zai Lab Limited (...
3,ZLAB,2021-03-01,07:00AM,Zai Lab Announces Financial Results for Second...
4,ZLAB,2021-02-27,09:23AM,15 Fastest Growing Biotech Companies in the US


In [12]:
# Sentiment scores dictionaries
zlab_sent = {
    "zlab_compound": [],
    "zlab_sentiment": [],
}

# Get sentiment for the tweets
for index, row in df.iterrows():
    try:
        # Sentiment scoring with VADER
        zlab_sentiment = analyzer.polarity_scores(row["title"])
        zlab_sent["zlab_compound"].append(zlab_sentiment["compound"])
        zlab_sent["zlab_sentiment"].append(get_sentiment(zlab_sentiment["compound"]))

    except AttributeError:
        pass

# Attaching sentiment columns to the News DataFrame
zlab_sentiment_df = pd.DataFrame(zlab_sent)
zlab_df = df.join(zlab_sentiment_df)

In [13]:
zlab_df.head()

,ticker,date,time,title,zlab_compound,zlab_sentiment
0,ZLAB,2021-03-12,11:54AM,7 Biotech Stocks With Catalysts That Go Far Be...,0.0000,0
1,ZLAB,2021-03-04,04:19PM,Why Amgen's $1.9 Billion Takeover Might Only W...,0.0000,0
2,ZLAB,2021-03-02,11:56PM,Analysts Are More Bearish On Zai Lab Limited (...,-0.2263,-1
3,ZLAB,2021-03-01,07:00AM,Zai Lab Announces Financial Results for Second...,0.0000,0
4,ZLAB,2021-02-27,09:23AM,15 Fastest Growing Biotech Companies in the US,0.1779,1


In [15]:
zlab_df.to_csv("../Data/zlab_combined_sentiment.csv", index=False)